我想做一个模型，基于transformer架构, 具体的细节:
1. 输入三个2维点集 A,B,C
2. 每个点构成一个2维token，坐标x, y
3. 每个点集长度固定为100,不够则padding (0,0) token;
4. 由transformer作为主体模块的编码器E，输入一个点集S（固定长度为100的2维token),输出一个50维的embedding向量T
5. 将 A\B\C 3个点集分别输入E得到对应的3个embedding向量Sa\Sb\Sc
6. 最终的损失函数为： 最小化 Sa - (Sb + Sc)及正则化惩罚项
7. 整体思路如上，该模型需要学习一个点集的编码方式，使得 原始点集 A=B+C(B和C是关于A的互补点集)的降维embedding向量也能反映这种关系

请提出好的建议并给出代码实现

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

class TransformerEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, num_heads, dropout):
        super().__init__()
        # 定义一个线性层，用于将输入的2D点集映射到hidden_dim维度的向量
        self.embedding = nn.Linear(input_dim, hidden_dim)
        # 定义一个Transformer编码器，用于将输入的向量序列编码成一个向量
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(hidden_dim, num_heads, hidden_dim, dropout),
            num_layers
        )
        # 定义一个线性层，用于将Transformer编码器的输出映射到output_dim维度的向量
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # 将输入的2D点集映射到hidden_dim维度的向量
        x = self.embedding(x)
        # 将batch_size放在第二个维度上
        x = x.permute(1, 0, 2)
        # 将输入的向量序列编码成一个向量
        x = self.transformer(x)
        # 将batch_size放回第一个维度上
        x = x.permute(1, 0, 2)
        # 将Transformer编码器的输出映射到output_dim维度的向量
        x = self.fc(x[:, 0, :])
        return x

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # 定义一个包含三个Transformer编码器的模型
        self.encoder = TransformerEncoder(2, 128, 50, 6, 8, 0.1)

    def forward(self, A, B, C):
        # 对A进行一个随机的仿射变换，得到点集B'
        B_ = A @ torch.randn(2, 2) + torch.randn(1, 2)
        # 随机去掉B'其中一些点（加入点集C）
        indices = torch.randperm(B_.shape[0])[:int(B_.shape[0] * 0.9)]
        C_ = B_[indices]
        B_ = B_[~indices]
        # 随机增加一些围绕B'中剩下点的噪声点（满足一定比率10%左右）
        noise = torch.randn(int(B_.shape[0] * 0.1), 2) + B_.mean(dim=0)
        B_ = torch.cat([B_, noise], dim=0)
        C_ = torch.cat([C_, noise], dim=0)
        # 将三个点集分别输入到Transformer编码器中，得到对应的三个embedding向量
        Sa = self.encoder(A)
        Sb = self.encoder(B_)
        Sc = self.encoder(C_)
        # 计算损失函数
        loss = torch.norm(Sa - (Sb + Sc)) + 0.01 * (torch.norm(Sa) + torch.norm(Sb) + torch.norm(Sc))
        return loss

# 构造数据集
data = torch.randn(100, 2)
dataset = MyDataset(data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 定义模型和优化器
model = MyModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
for epoch in range(10):
    for batch in dataloader:
        A = batch
        B = None
        C = None
        loss = model(A, B, C)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# 测试模型
A = torch.randn(100, 2)
B = None
C = None
with torch.no_grad():
    Sa = model.encoder(A)